In [ ]:
from functools import reduce

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_formats = ['svg']

In [ ]:
def read_all(file):
    names = ['Time', 'Type'] + ['col'+str(x) for x in range(1,10)]
    df = pd.read_csv(file, sep='\t', comment='#', header=None, names=names)
    return df

In [ ]:
def read_trace(path_trace_file):
    df = read_all(path_trace_file)
    df.Time = (df.Time - df.Time[0]) / 1000

    acc = df[df.Type == 'TYPE_ACCELEROMETER'].dropna(axis=1).rename(columns={'col1': 'a_x', 'col2': 'a_y', 'col3': 'a_z'}).drop('Type', axis=1).reset_index(drop=True)
    gyro = df[df.Type == 'TYPE_GYROSCOPE'].dropna(axis=1).rename(columns={'col1': 'w_x', 'col2': 'w_y', 'col3': 'w_z'}).drop('Type', axis=1).reset_index(drop=True)
    rot = df[df.Type == 'TYPE_ROTATION_VECTOR'].dropna(axis=1).rename(columns={'col1': 'rot_x', 'col2': 'rot_y', 'col3': 'rot_z'}).drop('Type', axis=1).reset_index(drop=True)
    mag = df[df.Type == 'TYPE_MAGNETIC_FIELD'].dropna(axis=1).rename(columns={'col1': 'B_x', 'col2': 'B_y', 'col3': 'B_z'}).drop('Type', axis=1).reset_index(drop=True)
    waypoint =  df.loc[df.Type == 'TYPE_WAYPOINT', ['Time', 'col1', 'col2']].rename(columns={'col1': 'd_x', 'col2': 'd_y'}).reset_index(drop=True)
    # There is some ambiguity in these guys.
    # wifi = ...
    # beacon = ...

    dfs = [acc, gyro, rot, mag]
    df_final = reduce(lambda left, right: pd.merge(left, right, on='Time', how='left'), dfs)
    
    df_final = df_final.astype('float32')
    waypoint = waypoint.astype('float32')
    return df_final, waypoint

In [ ]:
path_trace_file = "../input/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5d1340ade5d8ea0008e38ff8.txt"
df1, df2 = read_trace(path_trace_file)

## Trace Features

In [ ]:
df1.head()

In [ ]:
yl = {'a': 'acceleration (m/s$^2$)', 'w': 'Angular Velocity (rad/s)', 'rot': 'Rotation Vector', 'B': 'B-field ($\mu$T)'}
for feature in ['a', 'w', 'rot', 'B']:
    sub_feature = ['Time'] + [x for x in df1.columns if x.startswith(feature)]
    df1[sub_feature].plot(x='Time', figsize=(12, 3), xlabel='Time (s)', ylabel=yl[feature])
plt.show()

## Waypoints

In [ ]:
df2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))
df2.plot(x='Time', y=['d_x', 'd_y'], marker='o', title='Time Evolution', ax=ax1, ylabel='Position (m)')
df2.plot(x='d_x', y='d_y', marker='o', xlabel='d_x (m)', ylabel='d_y (m)', title='2D Position', ax=ax2)
plt.show()